In [1]:
import requests
from io import BytesIO
import pandas as pd
from zipfile import ZipFile
import re
from dtype_diet import report_on_dataframe, optimize_dtypes
import os
from dataprep.clean import clean_headers
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = 'https://portal.inmet.gov.br/uploads/dadoshistoricos/2022.zip'
r = requests.get(path, verify = False)
files = ZipFile(BytesIO(r.content))
pattern = re.compile("Porto Alegre", flags = re.IGNORECASE)
arquivo = []
for file in files.namelist():
    if re.search(pattern, str(file)):
        arquivo.append(file)
temp_poa = pd.read_csv(files.open(arquivo[0]), encoding="latin1", sep = ";", skiprows=8)

temp_poa_desc = pd.read_csv(files.open(arquivo[0]), encoding="latin1", sep = ";", nrows=7, header = None)
desc = {line[1][0]: line[1][1] for line in temp_poa_desc.iterrows()}

In [3]:
temp_poa.head()

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2022/01/01,0000 UTC,0,"1000,3","1000,3","999,4",NaN,25,"19,7","26,4",25,"19,7","19,3",72.0,65.0,72.0,126.0,"5,4","2,1",NaN
1,2022/01/01,0100 UTC,0,"1000,6","1000,6","1000,2",NaN,"24,1","19,8",25,"24,1","19,8","19,6",77.0,72.0,77.0,95.0,"4,3","1,3",NaN
2,2022/01/01,0200 UTC,0,"1000,5","1000,8","1000,5",NaN,"23,6","19,9","24,1","23,6","19,9","19,8",80.0,77.0,80.0,76.0,"3,4",1,NaN
3,2022/01/01,0300 UTC,0,1000,"1000,5",1000,NaN,"23,3","19,6","23,8","23,3","19,9","19,5",80.0,78.0,79.0,104.0,"2,6",1,NaN
4,2022/01/01,0400 UTC,0,"999,7",1000,"999,6",NaN,23,"20,1","23,6","22,5","20,2","19,4",85.0,78.0,84.0,132.0,"3,4","1,4",NaN


In [4]:
temp_poa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 20 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Data                                                   8760 non-null   object 
 1   Hora UTC                                               8760 non-null   object 
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       8730 non-null   object 
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  8730 non-null   object 
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        8729 non-null   object 
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       8729 non-null   object 
 6   RADIACAO GLOBAL (Kj/m²)                                4702 non-null   object 
 7   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           8730 non-null   object 
 8   TEMPERATURA DO PONTO DE ORVALHO (°C)            

In [5]:
proposed_df = report_on_dataframe(temp_poa, unit="MB")
new_df = optimize_dtypes(temp_poa, proposed_df)
print(f'Original df memory: {temp_poa.memory_usage(deep=True).sum()/1024/1024} MB')
print(f'Propsed df memory: {new_df.memory_usage(deep=True).sum()/1024/1024} MB')

Original df memory: 7.92751407623291 MB
Propsed df memory: 0.9679117202758789 MB


In [6]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 20 columns):
 #   Column                                                 Non-Null Count  Dtype   
---  ------                                                 --------------  -----   
 0   Data                                                   8760 non-null   category
 1   Hora UTC                                               8760 non-null   category
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       8730 non-null   category
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  8730 non-null   category
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        8729 non-null   category
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       8729 non-null   category
 6   RADIACAO GLOBAL (Kj/m²)                                4702 non-null   category
 7   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           8730 non-null   category
 8   TEMPERATURA DO PONTO DE ORVALHO (°C)  

In [7]:
desc

{'REGIAO:': 'S',
 'UF:': 'RS',
 'ESTACAO:': 'PORTO ALEGRE - JARDIM BOTANICO',
 'CODIGO (WMO):': 'A801',
 'LATITUDE:': '-30,05361111',
 'LONGITUDE:': '-51,17472221',
 'ALTITUDE:': '41,18'}

In [ ]:
# df = pd.DataFrame()
# for ano in range(2000, 2024):
#     path = f'https://portal.inmet.gov.br/uploads/dadoshistoricos/{ano}.zip'
#     r = requests.get(path, verify = False)
#     files = ZipFile(BytesIO(r.content))
#     arquivos = []
#     for file in files.namelist():
#         if file[-4:].lower() == ".csv":
#             arquivos.append(file)
#         else:
#             continue
#     df01 = pd.DataFrame()
#     for arquivo in arquivos:
#         info = pd.read_csv(files.open(arquivo), sep = ";", encoding = "latin-1", nrows=7, header = None)
#         desc = {line[1][0]: line[1][1] for line in temp_poa_desc.iterrows()}
#         df02 = pd.read_csv(files.open(arquivo),  sep = ";", encoding = "latin-1", skiprows = 8)
#         df02["ESTACAO"] = desc['ESTACAO:']
#         df02["UF"] = desc['UF:']
#         df02["REGIAO"] = desc['REGIAO:']
#         #nome_arquivo = arquivo.split('/')[1]
#         #df02.to_parquet(f'../../../data/inmet/inmet_{nome_arquivo[:-4]}.parquet')
#         df01 = pd.concat([df01, df02])
#     df = pd.concat([df, df01])

In [14]:
df = pd.DataFrame()
for ano in range(2000, 2024):
    print(ano)
    path = f'https://portal.inmet.gov.br/uploads/dadoshistoricos/{ano}.zip'
    r = requests.get(path, verify = False)
    files = ZipFile(BytesIO(r.content))
    arquivos = [file for file in files.namelist() if file.lower().endswith(".csv")]

    df01 = pd.DataFrame()
    nomes_colunas = []
    for arquivo in arquivos:
        info = pd.read_csv(files.open(arquivo), sep = ";", encoding = "latin-1", nrows=7, header = None)
        info2 = {line[1][0]: line[1][1] for line in temp_poa_desc.iterrows()}
        df02 = pd.read_csv(files.open(arquivo),  sep = ";", encoding = "latin-1", skiprows = 8, nrows=1)
        df02["ESTACAO"] = info2['ESTACAO:']
        df02["UF"] = info2['UF:']
        df02["REGIAO"] = info2['REGIAO:']
        df01 = pd.concat([df01, df02])
        for col in df02.columns.to_list():
            nomes_colunas.append(col)
    df = pd.concat([df, df01])

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8893 entries, 0 to 0
Data columns (total 26 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   DATA (YYYY-MM-DD)                                      5994 non-null   object 
 1   HORA (UTC)                                             5994 non-null   object 
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       8030 non-null   object 
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  8206 non-null   object 
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        8202 non-null   object 
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       8202 non-null   object 
 6   RADIACAO GLOBAL (KJ/m²)                                6036 non-null   object 
 7   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           8202 non-null   object 
 8   TEMPERATURA DO PONTO DE ORVALHO (°C)               

In [29]:
# Colunas com escritas diferentes:
    # DATA (YYYY-MM-DD) <-> Data
    # HORA (UTC) <-> Hora UTC
    # RADIACAO GLOBAL (Kj/m²) <-> RADIACAO GLOBAL (KJ/m²) 
df.isna().sum()

DATA (YYYY-MM-DD)                                        2899
HORA (UTC)                                               2899
PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                          863
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)     687
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)           691
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)          691
RADIACAO GLOBAL (KJ/m²)                                  2857
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)              691
TEMPERATURA DO PONTO DE ORVALHO (°C)                      779
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)                695
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)                695
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)          780
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)          781
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                  775
UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)                  777
UMIDADE RELATIVA DO AR, HORARIA (%)                       774
VENTO, D

In [30]:
df.nuni

DATA (YYYY-MM-DD)                                         493
HORA (UTC)                                                  1
PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                           78
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)    1423
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)          1420
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)         1409
RADIACAO GLOBAL (KJ/m²)                                    81
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)              204
TEMPERATURA DO PONTO DE ORVALHO (°C)                      188
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)                211
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)                204
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)          194
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)          199
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                   85
UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)                   87
UMIDADE RELATIVA DO AR, HORARIA (%)                        84
VENTO, D

In [24]:
df.sample(5)

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),...,"VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19,ESTACAO,UF,REGIAO,Data,Hora UTC,RADIACAO GLOBAL (Kj/m²)
0,NaN,NaN,NaN,"986,9","986,9","986,7","5138,1","24,7","18,8","24,7",...,93.0,10,"4,5",NaN,PORTO ALEGRE - JARDIM BOTANICO,RS,S,2019/01/01,0000 UTC,NaN
0,2012-01-01,00:00,0,"1010,5","1010,5","1009,1",-9999,"18,7","16,3","18,7",...,45,"15,6","4,8",NaN,PORTO ALEGRE - JARDIM BOTANICO,RS,S,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,PORTO ALEGRE - JARDIM BOTANICO,RS,S,2021/01/01,0000 UTC,NaN
0,2017-01-01,00:00,0,961,961,"960,7",-9999,"23,3","15,4","24,7",...,80,"2,2",",7",NaN,PORTO ALEGRE - JARDIM BOTANICO,RS,S,NaN,NaN,NaN
0,2013-01-01,00:00,0,"1002,9","1002,9","1001,9",-9999,"24,9","23,8",25,...,43,"1,1",0,NaN,PORTO ALEGRE - JARDIM BOTANICO,RS,S,NaN,NaN,NaN


In [26]:
#df.to_excel("inmet_database_sample.xlsx", index=False)

In [31]:
df01.drop("Unnamed: 19", axis=1, inplace=True)

In [32]:
colunas_antigas = df.columns
colunas_ajustadas = clean_headers(df).columns.to_list()
#df01.columns = colunas_ajustadas

Column Headers Cleaning Report:
	26 values cleaned (100.0%)


In [33]:
c_pairs = list(zip(colunas_antigas, colunas_ajustadas))

In [36]:
{antes: depois for antes, depois in c_pairs}

{'DATA (YYYY-MM-DD)': 'data_yyyy_mm_dd',
 'HORA (UTC)': 'hora_utc',
 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'precipitacao_total_horario_mm',
 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao_atmosferica_ao_nivel_da_estacao_horaria_m_b',
 'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'pressao_atmosferica_max_na_hora_ant_aut_m_b',
 'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'pressao_atmosferica_min_na_hora_ant_aut_m_b',
 'RADIACAO GLOBAL (KJ/m²)': 'radiacao_global_kj_m',
 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temperatura_do_ar_bulbo_seco_horaria_c',
 'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'temperatura_do_ponto_de_orvalho_c',
 'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'temperatura_maxima_na_hora_ant_aut_c',
 'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'temperatura_minima_na_hora_ant_aut_c',
 'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': 'temperatura_orvalho_max_na_hora_ant_aut_c',
 'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': 'tem

In [2]:
columns_dict = {
    'DATA (YYYY-MM-DD)': 'data',
    'Data': 'data',
    'HORA (UTC)': 'hora',
    'Hora UTC': 'hora',
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'precipitacao_total_horario_mm',
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'precipitacao_total_horario_mm',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao_atmosferica_ao_nivel_da_estacao_horaria_m_b',
    'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'pressao_atmosferica_max_na_hora_ant_aut_m_b',
    'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'pressao_atmosferica_min_na_hora_ant_aut_m_b',
    'RADIACAO GLOBAL (KJ/m²)': 'radiacao_global_kj_m',
    'RADIACAO GLOBAL (Kj/m²)': 'radiacao_global_kj_m',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temperatura_do_ar_bulbo_seco_horaria_c',
    'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'temperatura_do_ponto_de_orvalho_c',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'temperatura_maxima_na_hora_ant_aut_c',
    'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'temperatura_minima_na_hora_ant_aut_c',
    'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': 'temperatura_orvalho_max_na_hora_ant_aut_c',
    'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': 'temperatura_orvalho_min_na_hora_ant_aut_c',
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'umidade_rel_max_na_hora_ant_aut_%',
    'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'umidade_rel_min_na_hora_ant_aut_%',
    'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade_relativa_do_ar_horaria_%',
    'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'vento_direcao_horaria_gr_gr',
    'VENTO, RAJADA MAXIMA (m/s)': 'vento_rajada_maxima_m_s',
    'VENTO, VELOCIDADE HORARIA (m/s)': 'vento_velocidade_horaria_m_s',
    'ESTACAO': 'estacao',
    'UF': 'uf',
    'REGIAO': 'regiao',
}

In [41]:
df.rename(columns=columns_dict).sample(2)

,data,hora,precipitacao_total_horario_mm,pressao_atmosferica_ao_nivel_da_estacao_horaria_m_b,pressao_atmosferica_max_na_hora_ant_aut_m_b,pressao_atmosferica_min_na_hora_ant_aut_m_b,radiacao_global_kj_m,temperatura_do_ar_bulbo_seco_horaria_c,temperatura_do_ponto_de_orvalho_c,temperatura_maxima_na_hora_ant_aut_c,...,vento_direcao_horaria_gr_gr,vento_rajada_maxima_m_s,vento_velocidade_horaria_m_s,Unnamed: 19,estacao,uf,regiao,data,hora,radiacao_global_kj_m
0,2013-01-01,00:00,0,"902,7","902,7","901,6",-9999,"20,1","19,1","20,7",...,6,"8,5",4,NaN,PORTO ALEGRE - JARDIM BOTANICO,RS,S,NaN,NaN,NaN
0,2016-01-01,00:00,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,...,-9999,-9999,-9999,NaN,PORTO ALEGRE - JARDIM BOTANICO,RS,S,NaN,NaN,NaN


In [89]:
df["HORA (UTC)"].unique()

array(['00:00', nan], dtype=object)

In [95]:
df["Hora UTC"].sample()

0    0000 UTC
Name: Hora UTC, dtype: object

# TODO: ajustar decimal (for coluna replace "," "." etc)

In [103]:
df["PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"].replace(",",".", regex=True).astype("float")

0   -9999.0
0   -9999.0
0   -9999.0
0   -9999.0
0   -9999.0
      ...  
0    1013.5
0    1014.8
0     917.6
0     934.9
0     906.5
Name: PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB), Length: 8893, dtype: float64

# loop

In [ ]:
columns_dict = {
    'DATA (YYYY-MM-DD)': 'data',
    'Data': 'data',
    'HORA (UTC)': 'hora',
    'Hora UTC': 'hora',
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'precipitacao_total_horario_mm',
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'precipitacao_total_horario_mm',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao_atmosferica_ao_nivel_da_estacao_horaria_m_b',
    'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'pressao_atmosferica_max_na_hora_ant_aut_m_b',
    'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'pressao_atmosferica_min_na_hora_ant_aut_m_b',
    'RADIACAO GLOBAL (KJ/m²)': 'radiacao_global_kj_m',
    'RADIACAO GLOBAL (Kj/m²)': 'radiacao_global_kj_m',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temperatura_do_ar_bulbo_seco_horaria_c',
    'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'temperatura_do_ponto_de_orvalho_c',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'temperatura_maxima_na_hora_ant_aut_c',
    'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'temperatura_minima_na_hora_ant_aut_c',
    'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': 'temperatura_orvalho_max_na_hora_ant_aut_c',
    'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': 'temperatura_orvalho_min_na_hora_ant_aut_c',
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'umidade_rel_max_na_hora_ant_aut_%',
    'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'umidade_rel_min_na_hora_ant_aut_%',
    'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade_relativa_do_ar_horaria_%',
    'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'vento_direcao_horaria_gr_gr',
    'VENTO, RAJADA MAXIMA (m/s)': 'vento_rajada_maxima_m_s',
    'VENTO, VELOCIDADE HORARIA (m/s)': 'vento_velocidade_horaria_m_s',
    'ESTACAO': 'estacao',
    'UF': 'uf',
    'REGIAO': 'regiao',
}

In [3]:
col_types = {
    'precipitacao_total_horario_mm': float,
    'pressao_atmosferica_ao_nivel_da_estacao_horaria_m_b': float,
    'pressao_atmosferica_max_na_hora_ant_aut_m_b': float,
    'pressao_atmosferica_min_na_hora_ant_aut_m_b': float,
    'radiacao_global_kj_m': float,
    'temperatura_do_ar_bulbo_seco_horaria_c': float,
    'temperatura_do_ponto_de_orvalho_c': float,
    'temperatura_maxima_na_hora_ant_aut_c': float,
    'temperatura_minima_na_hora_ant_aut_c': float,
    'temperatura_orvalho_max_na_hora_ant_aut_c': float,
    'temperatura_orvalho_min_na_hora_ant_aut_c':float,
    'umidade_rel_max_na_hora_ant_aut_%': float,
    'umidade_rel_min_na_hora_ant_aut_%': float,
    'umidade_relativa_do_ar_horaria_%': float,
    'vento_direcao_horaria_gr_gr': float,
    'vento_rajada_maxima_m_s': float,
    'vento_velocidade_horaria_m_s': float,
    'estacao': 'category',
    'uf': 'category',
    'regiao': 'category'
    }

#df = df.astype(col_types)

In [9]:
df = pd.DataFrame()
for ano in range(2000, 2024):
    print(ano)
    path = f'https://portal.inmet.gov.br/uploads/dadoshistoricos/{ano}.zip'
    r = requests.get(path, verify = False)
    files = ZipFile(BytesIO(r.content))
    arquivos = [file for file in files.namelist() if file.lower().endswith(".csv")]

    df01 = pd.DataFrame()
    nomes_colunas = []
    for arquivo in arquivos:
        info = pd.read_csv(files.open(arquivo), sep = ";", encoding = "latin-1", nrows=7, header = None)
        info2 = {line[1][0]: line[1][1] for line in info.iterrows()}
        #df02 = pd.read_csv(files.open(arquivo),  sep = ";", encoding = "latin-1", skiprows = 8, nrows=1)
        df02 = pd.read_csv(files.open(arquivo),  sep = ";", encoding = "latin-1", skiprows = 8)
        df02.rename(columns=columns_dict, inplace=True)
        #df02["estacao"] = info2['ESTACAO:']
        df02["estacao"] = info.iloc[2,1]
        df02["uf"] = info.iloc[1,1]
        df02["regiao"] = info.iloc[0,1]
        #print(info.iloc[2,1], info.iloc[1,1], info.iloc[0,1])
        for col in df02.columns:
            df02.loc[:,col] = df02.loc[:,col].replace(",",".", regex=True)
        df02 = df02.astype(col_types)
        df01 = pd.concat([df01, df02])
        #for col in df02.columns.to_list():
        #    nomes_colunas.append(col)
    df = pd.concat([df, df01])

2000
2001
2002
2003


In [152]:
df.memory_usage(deep=True).sum()/1024/1024 #MB

In [ ]:
def ajusta_hora(x):
    if ":" in x:
        y = x
    elif "UTC" in x:
        y = x[:2] + ":" + x[2:4]
    else:
        y = None
    return y

def converte_data(x):
    try:
        y = pd.to_datetime(x, format = '%Y-%m-%d %H:%M:%S')
    except:
        y = pd.to_datetime(x, format = '%Y/%m/%d %H:%M:%S')
    return y

df_typed.loc[:, "hora"] = df_typed.loc[:, "hora"].apply(ajusta_hora)
df_typed["data_hora"] = df_typed["data"] + " " + df_typed["hora"]
df_typed.loc[:, "data_hora"] = df_typed.loc[:, "data_hora"].apply(converte_data) 

In [ ]:
df_typed.drop(["data_hora" "Unnamed: 19"], axis=1, inplace=True)
df_typed.set_index("data_hora",inplace=True)


KeyError: "['data', 'hora'] not found in axis"

In [ ]:
df_typed.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8893 entries, 2000-05-07 to 2023-01-01
Data columns (total 21 columns):
 #   Column                                               Non-Null Count  Dtype   
---  ------                                               --------------  -----   
 0   precipitacao_total_horario_mm                        8030 non-null   float64 
 1   pressao_atmosferica_ao_nivel_da_estacao_horaria_m_b  8206 non-null   float64 
 2   pressao_atmosferica_max_na_hora_ant_aut_m_b          8202 non-null   float64 
 3   pressao_atmosferica_min_na_hora_ant_aut_m_b          8202 non-null   float64 
 4   radiacao_global_kj_m                                 6224 non-null   float64 
 5   temperatura_do_ar_bulbo_seco_horaria_c               8202 non-null   float64 
 6   temperatura_do_ponto_de_orvalho_c                    8114 non-null   float64 
 7   temperatura_maxima_na_hora_ant_aut_c                 8198 non-null   float64 
 8   temperatura_minima_na_hora_ant_aut_c    

In [ ]:
proposed_df = report_on_dataframe(df_typed.reset_index(), unit="MB")
new_df = optimize_dtypes(df_typed.reset_index(), proposed_df)

print(f'Original df memory: {df_typed.reset_index().memory_usage(deep=True).sum()/1024/1024} MB')
print(f'Propsed df memory: {new_df.memory_usage(deep=True).sum()/1024/1024} MB')

Original df memory: 1.3151845932006836 MB
Propsed df memory: 1.1625261306762695 MB


In [ ]:
new_df.info()

,Current dtype,Proposed dtype,Current Memory (MB),Proposed Memory (MB),Ram Usage Improvement (MB),Ram Usage Improvement (%)
Column,,,,,,
data_hora,datetime64[ns],None,34.800781,NaN,NaN,NaN
precipitacao_total_horario_mm,float64,None,34.800781,NaN,NaN,NaN
pressao_atmosferica_ao_nivel_da_estacao_horaria_m_b,float64,None,34.800781,NaN,NaN,NaN
pressao_atmosferica_max_na_hora_ant_aut_m_b,float64,None,34.800781,NaN,NaN,NaN
pressao_atmosferica_min_na_hora_ant_aut_m_b,float64,None,34.800781,NaN,NaN,NaN
radiacao_global_kj_m,float64,None,34.800781,NaN,NaN,NaN
temperatura_do_ar_bulbo_seco_horaria_c,float64,None,34.800781,NaN,NaN,NaN
temperatura_do_ponto_de_orvalho_c,float64,None,34.800781,NaN,NaN,NaN
temperatura_maxima_na_hora_ant_aut_c,float64,None,34.800781,NaN,NaN,NaN


In [ ]:
df.to_parquet("inmet_data.parquet")